In [5]:
# coding: utf-8
"""
igraphを使ったグラフクラスタリング
greedyアルゴリズム（louvain法）によってクラスタリングした後
サブグラフの中心性指標を図ることで単語の順位付けを行う
媒介中心性と固有ベクトル中心性の２種類を採用

d3.jsによる可視化をするための下準備
階層的なクラスタを全て抜き取る
"""
from igraph import *
import csv
import collections
import pickle
import numpy as np
import cairo

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

# 有向エッジリストを入力して、重み付き無向ネットワークを出力する
def cal_edgelist_to_network(list_edge):
    # 有向エッジリストを無向エッジリストに変換する
    list_edge = [tuple(sorted(row)) for row in list_edge]
    # ノードリスト
    list_vertices = list(set([word for row in list_edge for word in row]))
    # エッジリストとそのweightを作成
    tuple_edge, tuple_weight = zip(*collections.Counter(list_edge).items())
    return {"vertex": list_vertices, "edge": list(tuple_edge), "weight": list(tuple_weight)}

# クラスタリング済みのネットワークを元にサブグラフのリストを作成
# vertexには全てのvertexを代入する（PageRankを計算するため）
def cal_cluster_to_network(dict_network):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # クラスタごとにwordをまとめる
    dict_cluster = collections.defaultdict(list)
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_cluster[cluster].append(word)

    # リストに変換
    list_cluster_vertex = [row[1] for row in dict_cluster.items()]
    
    # 同様にエッジとウェイトのリストも作成する
    list_cluster_edge = []
    list_cluster_weight = []
    for cluster_vertex in list_cluster_vertex:
        list_cluster_edge_one = []
        list_cluster_weight_one = []
        # エッジリストの中に、一つでもノードが含まれていれば、そのクラスのノードに含める
        for row, weight in zip(dict_network["edge"], dict_network["weight"]):
            # and と or を切り替えることによって性能の比較
            if row[0] in cluster_vertex or row[1] in cluster_vertex:
                list_cluster_edge_one.append(row)
                list_cluster_weight_one.append(weight)
        list_cluster_edge.append(list_cluster_edge_one)
        list_cluster_weight.append(list_cluster_weight_one)
    
    # まとめる
    list_dict_network = [{"vertex": dict_network["vertex"],
                          "edge": cluster_edge,
                          "weight": cluster_weight}
                         for cluster_edge, cluster_weight
                         in zip(list_cluster_edge, list_cluster_weight)]
    
    return list_dict_network

# f_measureを計算する
def cal_f_measure(list_predict_measure):
    # 生成したクラスタ内のカウント
    dict_predict_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_predict_cluster[row[0]].append(row[1])
        
    # もとあるクラス内のカウント
    dict_measure_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_measure_cluster[row[1]].append(row[0])
    
    # local_purityの計算
    list_purity = []
    for row in dict_predict_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_purity.append([major_class, class_num])
    purity = float(np.sum(zip(*list_purity)[0])) / np.sum(zip(*list_purity)[1])
    print purity
    
    # inverse_purityの計算
    list_inverse_purity = []
    for row in dict_measure_cluster.items():
        score1 = len(row[1])
        class_number = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][0]
        score2 = len(dict_predict_cluster[class_number])
        score3 = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        score4 = float(score3) * score2 / score1
        list_inverse_purity.append(score4)
    inverse_purity = np.sum(list_inverse_purity) / len(list_predict_measure)
    print inverse_purity
    
    return 2 / (1 / purity + 1 / inverse_purity)

エッジリストを読み込み、louvain法によりクラスタリングする

In [2]:
# エッジリストの読み込み
list_edge = readcsv("./files/list_edgelist20160127.csv")
# 元のネットワークを作成する（無向）
dict_network_master = cal_edgelist_to_network(list_edge)
# g = Graph(directed=True)
g_master = Graph()
g_master.add_vertices(dict_network_master["vertex"])
g_master.add_edges(dict_network_master["edge"])
# louvain法によるクラスタリング
result = g_master.community_fastgreedy(weights=dict_network_master["weight"])

クラスタ数:  7


階層クラスタリングの結果をもとに、dictionaryにクラスタid, マージしたクラスタ、ノード（重み付き）、エッジを格納する

In [91]:
dict_cluster = {}
# 初期クラスタのvertexを記録
for i, word in enumerate(dict_network_master["vertex"]):
    dict_cluster[i] = {"vertex": [word]}

# クラスタリング後のvertex、マージしたクラスタidを記録
for i, row in enumerate(result.merges):
    dict_cluster[i + len(dict_network_master["vertex"])] = {}
    dict_cluster[i + len(dict_network_master["vertex"])]["vertex"] = dict_cluster[row[0]]["vertex"] + dict_cluster[row[1]]["vertex"]
    dict_cluster[i + len(dict_network_master["vertex"])]["merge"] = row

# 各クラスタのvertexからedge、weightを抜き取り、pagerankを計算する
for i in range(len(dict_cluster)):
    list_cluster_edge_one = []
    list_cluster_weight_one = []
    list_cluster_vertex_one = []
    # エッジリストの中に、一つでもノードが含まれていれば、そのクラスのノードに含める
    for row, weight in zip(dict_network_master["edge"], dict_network_master["weight"]):
    # and と or を切り替えることによって性能の比較
        if row[0] in dict_cluster[i]["vertex"] or row[1] in dict_cluster[i]["vertex"]:
            list_cluster_edge_one.append(row)
            list_cluster_weight_one.append(weight)
            list_cluster_vertex_one.append(row[0])
            list_cluster_vertex_one.append(row[1])
    dict_cluster[i]["edge"] = list_cluster_edge_one
    dict_cluster[i]["weight"] = list_cluster_weight_one
    list_cluster_vertex_one = list(set(list_cluster_vertex_one))
    g_sub = Graph()
    g_sub.add_vertices(list_cluster_vertex_one)
    g_sub.add_edges(dict_cluster[i]["edge"])
    # pagerankを計算し、vertexと共に記録する
    result_pagerank = g_sub.pagerank(directed=False, weights=dict_cluster[i]["weight"])
    list_cluster_vertex_one = sorted(zip(list_cluster_vertex_one, result_pagerank), key=lambda x: x[1], reverse=True)
    dict_cluster[i]["vertex"] = list_cluster_vertex_one

dict_clusterの情報をJSON形式で保存する

TypeError: 'int' object has no attribute '__getitem__'